# Análisis Exploratorio de Datos (EDA) - Airbnb Barcelona
Trabajo práctico final - Fundamentos de Análisis de Datos en Python - Primer Cuatrimestre 2025 - Vázquez Nadal

## Introducción
En este trabajo se busca analizar datos de alojamientos de Airbnb en Barcelona. A partir de un dataset público, se aplican técnicas de exploración de datos para responder preguntas de negocio relevantes, identificar patrones y generar recomendaciones.

Las tres preguntas que se buscarán responder en este trabajo son las siguientes:

1- ¿Qué factores explican mejor el precio por noche de un alojamiento en Barcelona?

2- ¿Qué barrios de Barcelona ofrecen mejor relación calidad-precio según las reviews y los precios?

3- ¿Cómo afecta la condición de "superhost" y la cantidad de propiedades que administra un host al éxito del alojamiento (ingresos estimados o reviews)?


##  Carga y limpieza de datos

In [3]:
# 0.0 CARGA DE DATOS

import pandas as pd
from google.colab import drive

# Conexion a Google Drive
drive.mount('/content/drive')

# Abro el dataset
df = pd.read_csv('/content/drive/MyDrive/Electiva Python/listings.csv')

# Mostramos las primeras 5 filas para verificar que se hayan cargado correctamente
df.head()

# Funcion para renombrar columnas a camelCase
def to_camel_case(s):
    parts = s.replace('-', '_').replace('/', '_').split('_')
    return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:])

# Renombrar las columnas del dataset
df.columns = [to_camel_case(col) for col in df.columns]

# Mostrar las nuevas columnas para verificar
df.columns.tolist()

# 0.1 LIMPIEZA DE DATOS

# Seleccion de columnas relevantes para las 3 preguntas de analisis
columns_to_keep = [
    'id',
    'name',
    'description',
    'neighbourhoodCleansed',
    'latitude',
    'longitude',
    'propertyType',
    'roomType',
    'accommodates',
    'bedrooms',
    'beds',
    'price',
    'amenities',
    'minimumNights',
    'maximumNights',
    'availability365',
    'numberOfReviews',
    'reviewScoresRating',
    'reviewScoresAccuracy',
    'reviewScoresCleanliness',
    'reviewScoresCheckin',
    'reviewScoresCommunication',
    'reviewScoresLocation',
    'reviewScoresValue',
    'hostIsSuperhost',
    'hostTotalListingsCount',
    'estimatedRevenueL365d',
    'estimatedOccupancyL365d'
]

# Crear un nuevo dataset solo con las columnas seleccionadas
df_filtered = df[columns_to_keep].copy()

# Verificar los tipos de datos antes de la limpieza
df_filtered.dtypes

# Limpieza de la columna 'price': eliminar simbolos y convertir a float
df_filtered['price'] = df_filtered['price'].replace('[\$,]', '', regex=True).astype(float)

# Conversion de 'hostIsSuperhost' a booleano
df_filtered['hostIsSuperhost'] = df_filtered['hostIsSuperhost'].map({'t': True, 'f': False})

# Convertir a enteros las columnas que representan cantidades (cuando no tengan nulos)
df_filtered['bedrooms'] = df_filtered['bedrooms'].fillna(0).astype(int)
df_filtered['beds'] = df_filtered['beds'].fillna(0).astype(int)
df_filtered['hostTotalListingsCount'] = df_filtered['hostTotalListingsCount'].fillna(0).astype(int)

# Completar valores nulos en reviewScores con -1 para identificar que no hay datos
review_cols = [col for col in df_filtered.columns if col.startswith('reviewScores')]
df_filtered[review_cols] = df_filtered[review_cols].fillna(-1)

# Mostrar resumen final de tipos y cantidad de valores nulos
df_filtered.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19422 entries, 0 to 19421
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         19422 non-null  int64  
 1   name                       19422 non-null  object 
 2   description                18676 non-null  object 
 3   neighbourhoodCleansed      19422 non-null  object 
 4   latitude                   19422 non-null  float64
 5   longitude                  19422 non-null  float64
 6   propertyType               19422 non-null  object 
 7   roomType                   19422 non-null  object 
 8   accommodates               19422 non-null  int64  
 9   bedrooms                   19422 non-null  int64  
 10  beds                       19422 non-null  int64  
 11  price                      15273 non-null  float64
 12  amenities                  19422 non-null  object 
 13  minimumNights       

##  EDA - 1. Factores que explican el precio

In [4]:
# Filtramos solo filas con precio valido (>0)
df_price = df_filtered[df_filtered['price'] > 0].copy()

# Variables numericas a explorar contra el precio
num_vars = [
    'accommodates', 'bedrooms', 'beds', 'availability365',
    'reviewScoresRating', 'reviewScoresAccuracy', 'reviewScoresCleanliness',
    'reviewScoresLocation', 'reviewScoresValue'
]

# Calcular correlaciones con el precio
correlations = df_price[num_vars + ['price']].corr()['price'].sort_values(ascending=False)

# Mostrar top 5 correlaciones con el precio
correlations.head(6)

,price
price,1.000000
bedrooms,0.320842
accommodates,0.318546
beds,0.249093
availability365,0.057383
reviewScoresCleanliness,-0.012081


##  EDA - 2. Relación calidad/precio por barrio

In [5]:
# Usamos 'reviewScoresRating' como proxy de calidad y 'price' como proxy de costo

# Filtrar registros con datos validos
df_barrio = df_price[
    (df_price['reviewScoresRating'] > 0) &
    (df_price['price'] > 0)
].copy()

# Calcular media de review y precio por barrio
barrio_stats = df_barrio.groupby('neighbourhoodCleansed').agg({
    'price': 'mean',
    'reviewScoresRating': 'mean',
    'id': 'count'
}).rename(columns={'id': 'listingCount'}).reset_index()

# Crear columna con ratio calidad/precio
barrio_stats['qualityToPriceRatio'] = barrio_stats['reviewScoresRating'] / barrio_stats['price']

# Filtrar barrios con al menos 30 listings para mayor representatividad
barrio_stats = barrio_stats[barrio_stats['listingCount'] >= 30]

# Ordenar por mejor relacion calidad/precio
barrio_stats_sorted = barrio_stats.sort_values(by='qualityToPriceRatio', ascending=False)

barrio_stats_sorted.head()

,neighbourhoodCleansed,price,reviewScoresRating,listingCount,qualityToPriceRatio
52,la Marina de Port,62.857143,4.794000,35,0.076268
16,Sant Martí de Provençals,72.027027,4.479730,37,0.062195
35,el Congrés i els Indians,78.966667,4.733333,30,0.059941
58,la Sagrera,83.100000,4.721000,40,0.056811
32,el Carmel,82.112903,4.595968,62,0.055971


##  EDA - 3. Impacto del superhost y cantidad de propiedades

In [6]:
# Filtrar datos validos para el analisis
df_host = df_price[
    df_price['estimatedRevenueL365d'].notnull() &
    df_price['hostIsSuperhost'].notnull()
].copy()

# Agrupamos por si es superhost o no
superhost_stats = df_host.groupby('hostIsSuperhost').agg({
    'estimatedRevenueL365d': 'mean',
    'numberOfReviews': 'mean',
    'reviewScoresRating': 'mean',
    'id': 'count'
}).rename(columns={'id': 'listingCount'}).reset_index()

# Agrupamos por rango de cantidad de propiedades del host
df_host['hostSize'] = pd.cut(df_host['hostTotalListingsCount'], bins=[0, 1, 5, 10, 1000],
                             labels=['1 propiedad', '2-5 propiedades', '6-10 propiedades', '+10 propiedades'])

host_size_stats = df_host.groupby('hostSize').agg({
    'estimatedRevenueL365d': 'mean',
    'numberOfReviews': 'mean',
    'reviewScoresRating': 'mean',
    'id': 'count'
}).rename(columns={'id': 'listingCount'}).reset_index()

superhost_stats, host_size_stats

<ipython-input-6-5de66398c996>:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  host_size_stats = df_host.groupby('hostSize').agg({


(   hostIsSuperhost  estimatedRevenueL365d  numberOfReviews  \
 0            False           10277.231755        39.517817   
 1             True           25095.378604       109.924279   
 
    reviewScoresRating  listingCount  
 0            2.961989         11085  
 1            4.429014          3711  ,
            hostSize  estimatedRevenueL365d  numberOfReviews  \
 0       1 propiedad           15265.781250        86.379755   
 1   2-5 propiedades           14834.785453        77.362447   
 2  6-10 propiedades           17988.433787        74.811444   
 3   +10 propiedades           12542.985842        39.935433   
 
    reviewScoresRating  listingCount  
 0            3.637507          1472  
 1            3.766640          3286  
 2            3.922104          1835  
 3            2.970106          8193  )

# Bonus: Modelo predictivo simple de ingresos anuales

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

df_filtered = df[columns_to_keep].copy()
df_filtered['price'] = df_filtered['price'].replace('[\$,]', '', regex=True).astype(float)
df_filtered['hostIsSuperhost'] = df_filtered['hostIsSuperhost'].map({'t': True, 'f': False})
df_filtered['bedrooms'] = df_filtered['bedrooms'].fillna(0).astype(int)
df_filtered['beds'] = df_filtered['beds'].fillna(0).astype(int)
df_filtered['hostTotalListingsCount'] = df_filtered['hostTotalListingsCount'].fillna(0).astype(int)
review_cols = [col for col in df_filtered.columns if col.startswith('reviewScores')]
df_filtered[review_cols] = df_filtered[review_cols].fillna(-1)

# Filtramos registros con datos validos para el nuevo modelo
df_income = df_filtered[
    (df_filtered['estimatedRevenueL365d'].notnull()) &
    (df_filtered['reviewScoresRating'] > 0)
].copy()

# Seleccionamos features relevantes y target (ingresos anuales)
features = [
    'accommodates', 'bedrooms', 'beds', 'price', 'availability365',
    'reviewScoresRating', 'hostIsSuperhost', 'roomType', 'propertyType'
]
df_income = df_income[features + ['estimatedRevenueL365d']]

# Simplificamos propertyType (top 10 + Other)
top_props = df_income['propertyType'].value_counts().nlargest(10).index
df_income['propertyType'] = df_income['propertyType'].apply(lambda x: x if x in top_props else 'Other')

# Convertimos variables categoricas y booleanas
df_income['hostIsSuperhost'] = df_income['hostIsSuperhost'].fillna(False).astype(int)
df_encoded = pd.get_dummies(df_income, columns=['roomType', 'propertyType'], drop_first=True)

# Dividimos en X (features) e Y (target)
X = df_encoded.drop(columns='estimatedRevenueL365d')
y = df_encoded['estimatedRevenueL365d']

# Split train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones y metricas
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, r2

<ipython-input-7-fc9001fee534>:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_income['hostIsSuperhost'] = df_income['hostIsSuperhost'].fillna(False).astype(int)


(9424.198121454649, 0.43899155082050534)